In [1]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
import os, os.path
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime 
import ast

from sklearn.model_selection import train_test_split
import time
from geopy.geocoders import Nominatim 
from multiprocessing import Process
import dask
import dask.dataframe as dd


### Location to countries

In [2]:
def location_to_country(text):
    nm = Nominatim(user_agent="http")
    try:
        place, (lat, lng) = nm.geocode(text, timeout= 20)
    except Exception:
        return '-'
    else:
        return place.split(', ')[-1] 

In [3]:
def search_location(df_tweet,locDF=None,loadloc=False):
    locDict=dict()
    countrySet=set()
    if loadloc==True:
        for i,row in locDF.iterrows():
            locDict[row['loc']]=row['country']
            countrySet.add(row['country'])
    # print(locDict)
    # print(countrySet)
    nrows = df_tweet.shape[0]
    count=0
    timeStart=time.time()
    stageTime=time.time()
    localProcess=0
    stagelocal=0
    for i in range(nrows):
        if df_tweet['country'][i] != '-':
            
            text = df_tweet['country'][i]
            temp_str=text.split(',')[-1]
            if temp_str.lower() in countrySet:
                df_tweet['country'][i]=temp_str.lower()
                localProcess+=1
            elif locDict.get(text.lower()) != None:
                df_tweet['country'][i]=locDict.get(text.lower())
                localProcess += 1
            else:
                country = location_to_country(text)
                locDict[text.lower()]=country.lower()
                countrySet.add(country.lower())
                df_tweet['country'][i]=country.lower()

            # print(i)
        count +=1
        # if count % 100 ==0:
        #     print('process count:',count)
        #     print('Stage time use:',time.time()-stageTime)
        #     print('total time use:',time.time()-timeStart)
        #     print('local process:',localProcess)
        #     print('local process rate',(localProcess-stagelocal)/100)
        #     stageTime=time.time()
        #     stagelocal=localProcess
    
    return df_tweet, locDict

In [4]:
def en_ops(filename, output_string, directory):
    df = pd.read_csv(directory + "\\" + filename)
    print("read")
    df_keep = df[['created_at','id_str','text','user','lang','favorite_count','retweet_count', 'quote_count']]
    df_keep = df_keep[df_keep['text'].notna()].reset_index(drop=True)

    # Drop duplicated observations 
    if df_keep.duplicated().sum():
        df_keep.drop_duplicates(keep = 'first', inplace = True)

    df_keep.duplicated().sum()

    # Preprecessing Created_at
    len_date = len(df_keep['created_at'][0])
    df_keep = df_keep[df_keep['created_at'].apply(lambda x: len(str(x)) == len_date)].reset_index(drop=True)
    df_keep['created_at'] = df_keep['created_at'].apply(lambda x: datetime.strftime(datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H-%M-%S'))

    # Preprecessing Location 'nan' and 'None' are marked as '-'

    df_keep['location'] = df_keep['user'].copy(deep=True)
    df_keep['location'][df_keep['location'].isnull()] = '-'

    def pick_location(x):
        if x == '-':
            return x 
        else:
            return ast.literal_eval(x)['location']  

    df_keep['location'] = df_keep['location'].apply(lambda x: pick_location(x))

    if df_keep['location'].isnull().sum():
        df_keep['location'][df_keep['location'].isnull()] = '-'
    
    df_keep_en = df_keep[df_keep['lang']=='en'].reset_index(drop=True)
    df_keep_en = df_keep_en[df_keep_en['location']!='-'].reset_index(drop=True)
    df_keep_en['day'] = df_keep_en['created_at'].apply(lambda x: x.split(' ')[0])
    df_keep_en['hour'] = df_keep_en['created_at'].apply(lambda x: x.split(' ')[1])
    df_keep_en['hour'] = df_keep_en['hour'].apply(lambda x: x.split('-')[0])
    df_keep_en['hour'] = df_keep_en['day'] +'-'+ df_keep_en['hour']
    random_seed = 12
    df_keep_en, df_temp = train_test_split(df_keep_en, train_size=0.6, random_state=random_seed, stratify=df_keep_en['hour'])
    df_keep_en = df_keep_en.reset_index(drop=True)
    df_keep_en.drop(['day', 'hour'], axis=1, inplace=True)
    df_keep_en['country'] = df_keep_en['location'].copy(deep=True)
    nrows = df_keep_en.shape[0]
    loc=pd.read_csv(r'D:\covid research\src\local_exe\step 2\en_loc.csv')
    print("searching")
    df_keep_en, locDict=search_location(df_keep_en,locDF=loc,loadloc=True)
    print("writing to disk")
    df_keep_en.to_csv('D:/covid research/output/step 2 output/' + output_string + filename, index = False, encoding='utf-8-sig')
    # locDF=pd.DataFrame()
    # locDF['loc']=locDict.keys()
    # locDF['country']=locDict.values()
    # locDF.to_csv(r'D:\covid research\src\local_exe\en_loc.csv')

In [5]:
if __name__ == '__main__':
    
    
    directory = r'D:\covid research\output\step 1 output\2020-06-output'
    stuff_needs_processing = os.listdir(directory)
    output_string = "step2-output-EN-2020-06-"
    
    for each_file in stuff_needs_processing:
        try:
            en_ops(each_file, output_string, directory)
            print(f"finished processing {output_string + each_file}")
        except:
            print("there is something wrong with this file: ", each_file)
            logf = open("D:/covid research/output/step 2 output/log/2020-06-en-log.txt", "a")
            logf.write(each_file)
            logf.close()  

read
searching
writing to disk
finished processing step2-output-EN-2020-06-01.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-02.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-04.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-05.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-06.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-07.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-08.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-09.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-10.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-11.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-12.csv
read
searching
writing to disk
finished processing step2-output-EN-2020-06-13.csv
read
searching
w